# Infer ***soma*** -  1️⃣ 
(🚨🚨🚨🚨 Steps 2-9 depend on establishing a good solution here.)

--------------

## OBJECTIVE: ✅ Infer sub-cellular component soma in order to understand interactome 

>> WARNING:  THIS DOES NOT WORK WELL

To measure shape, position, and size of the cell body -- the soma.    There are a variety of signals from which we could make this inference.  The two most promising are a composite signal including the residual from linear unmixing (e.g. `ch = [1, 4, 5,7]`) and a signal derived from the lysosome channel (`ch = 1`).    In all procdures scaling the intensities to find the lower florescence signals at the edge of the soma against baseline is employed.  

In the long term we can build of a database of "ground truth" by sourcing additional markers which can be iteratively improved.  For example using the Allen Cell "Label Free" segmentation results should provide a good corroboration or constraints to the procedures outlined below.  

A first possible _workflow_ is illustrated below.






# IMPORTS

In [3]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc_2d.utils.file_io import (read_czi_image,
                                                                    read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)

from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH )                                                                    
from infer_subc_2d.utils.img import *

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

# IMAGE PROCESSING  OBJECTIVE :  infer ***soma***
 
> #### Note:  we are using the Nuclei of the brightest cell to aid in inferring the Soma and Cytosol objects.   Because we do NOT have a direct cell membrane / soma signal this is the trickiest and potentially problematic part of the overall sub-cellular component inference.   The Soma (via the Cytosol mask) will be used to define ALL subsequent sub-cellular Objects.

------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [4]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [5]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [6]:

ch_to_agg = ( LYSO_CH ,
                        MITO_CH ,
                        GOLGI_CH ,
                        PEROXI_CH ,
                        ER_CH ,
                        LIPID_CH )
                            
nuc_ch = NUC_CH
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 




## get the inferred nuclei object

(takes < 1 sec)

In [7]:


img_2D = img_data[:,[optimal_Z],:,:].copy()

# raw_nuclei = img_2D[0].copy()
# NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


# IMAGE PROCESSING PROTOTYPE, Objective 2:  infer ***soma***

>WORKFLOW #1  - modified MCZ 3/20

Segmentation on a 3 channel composite as per 3/20 pipeline from MCZ
Summary - Starting with a linear combination of three signals,  the signal is smoothed and non-linearly combined (logrithmic and edge detected) for thresholding. 
## summary of steps

➡️ INPUT
- multi-channel sum (4*1,5,7)
- labeled NUCLEI (objective #1)

PRE-PROCESSING
- ne-noise and somoothe
- log transform inensities
- scale to max 1.0
- create non-linear aggregate of log-intensity + scharr filtered 

CORE PROCESSING
- mask object segmentation at bottom

POST-PROCESSING
  - fill holes
  - remove small objects

POST-PROCESSING
  - keep only the "most intense" Soma


OUTPUT ➡️ 
- mask of SOMA


### INPUT

In [21]:

###################
# INPUT
###################
struct_img_raw = (4. * img_2D[LYSO_CH].copy() + 
                               1. * img_2D[ER_CH].copy() + 
                               1. * img_2D[RESIDUAL_CH].copy() )


nuc_img_raw = img_2D[NUC_CH].copy() 

In [39]:
def _raw_soma_MCZ(img_in):
    """ define soma image
    """
    SOMA_W = (4.,1.,1.)
    SOMA_CH = (LYSO_CH,ER_CH,RESIDUAL_CH) #LYSO, ER, RESIDUAL
    img_out = np.zeros_like(img_in[0]).astype(np.double)
    for w,ch in zip(SOMA_W,SOMA_CH):
        img_out += w*img_in[ch]
    return img_out

_struct_img_raw = _raw_soma_MCZ(img_2D)


### PRE-PROCESSING


In [40]:

###################
# PRE_PROCESSING
###################
#
# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0

med_filter_size = 15  


################# part 1
raw_soma_linear = min_max_intensity_normalization(struct_img_raw.copy())

struct_img = median_filter_slice_by_slice(raw_soma_linear, size=med_filter_size)


structure_img_smooth = ndi.gaussian_filter( struct_img,
                                                                                sigma=gaussian_smoothing_sigma,
                                                                                mode="nearest", 
                                                                                truncate=gaussian_smoothing_truncate_range,
                                                                                )


# NON-Linear aggregation
log_image, d = log_transform( structure_img_smooth ) 
log_image = intensity_normalization(  log_image,  scaling_param=[0] )

edges = filters.scharr(log_image)

composite_soma = intensity_normalization(  edges,  scaling_param=[0] ) + log_image 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [41]:
def _non_linear_soma_transform_MCZ(in_img):
    """ non-linear distortion to fill out soma
    log + edge of smoothed composite
    """


    # non-Linear processing
    log_img, d = log_transform( in_img.copy() ) 
    log_img = min_max_intensity_normalization(  log_img )
    return min_max_intensity_normalization(  filters.scharr(log_img) )  + log_img

_composite_soma = _non_linear_soma_transform_MCZ(structure_img_smooth)


In [42]:

################# part 2 Nuclei pre=process
nuc_img_raw = img_2D[0].copy() 
# median filter in 2D / convert to float 0-1.   get rid of the "residual"
nuclei = min_max_intensity_normalization(nuc_img_raw)

med_filter_size = 4   
# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
nuclei = median_filter_slice_by_slice( 
                                                                nuclei,
                                                                size=med_filter_size  )

gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0
nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
                                                                                            sigma=gaussian_smoothing_sigma,
                                                                                            truncate_range = gaussian_smoothing_truncate_range
                                                                                            )





### CORE PROCESSING

In [43]:
###################
# CORE_PROCESSING
###################
low_level_min_size =  100

################# part 1

# "Masked Object Thresholding" - 3D
bw, _bw_low_level = MO(composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
                                                

################# part 2 : nuclei thresholding
#struct_obj = struct_img > filters.threshold_li(struct_img)
threshold_factor = 0.9 #from cellProfiler
thresh_min = .1
thresh_max = 1.
NU_object = apply_log_li_threshold(nuclei, threshold_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)

hole_width = 5  
# # wrapper to remoce_small_objects
NU_object = remove_small_holes(NU_object, hole_width ** 2 )

small_object_max = 45
NU_object = size_filter_2D(NU_object, 
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)



NU_label = label(NU_object)



### POST-PROCESSING

In [44]:
###################
# POST_PROCESSING
###################

# 2D 
hole_width = 40
#removed_holes = remove_small_holes(bw, hole_width ** 2 )
removed_holes = hole_filling(bw, hole_min =0 , hole_max=hole_width**2, fill_2d = True) 


small_object_max = 45
cleaned_img = size_filter_2D(removed_holes, 
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)

# limit the labeling to where we have soma or NUclear signal
# watershed_mask = np.logical_or(cleaned_img, NU_labels > 0)
watershed_mask = cleaned_img 
inverted_img = 1. - composite_soma

labels_out = watershed(
            inverted_img,
            markers=NU_label,
            connectivity=np.ones((1, 3,3), bool),
            mask=watershed_mask,
            )



In [45]:
def _masked_inverted_watershed(img_in,markers, mask):
    """wrapper for watershed on inverted image and masked


    """
    watershed_mask = cleaned_img 
    inverted_img = 1. - composite_soma

    labels_out = watershed(
                1. - img_in,
                markers=markers,
                connectivity=np.ones((1, 3,3), bool),
                mask=mask,
                )
    return labels_out

_labels_out = _masked_inverted_watershed(composite_soma,NU_label, cleaned_img)




In [46]:
labels_out.shape, NU_label.shape

((1, 768, 768), (1, 768, 768))

In [47]:
###################
# POST- POST_PROCESSING
###################
# keep the "SOMA" label which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [ raw_soma_linear[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_signal)]
keep_label

soma_out = np.zeros_like(labels_out)
soma_out[labels_out==keep_label] = 1

In [48]:
def _choose_max_label(raw_signal: np.ndarray, labels_in: np.ndarray):
    """ keep only the label with the maximum raw signal

    """

    all_labels = np.unique(labels_in)[1:]

    total_signal = [ raw_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    labels_max = np.zeros_like(labels_out)
    labels_max[labels_out==keep_label] = 1
    return labels_max

_soma_out = _choose_max_label(raw_soma_linear,labels_out)

#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [49]:

viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )


In [50]:


viewer.scale_bar.visible = True
viewer.add_labels(
    labels_out,
    scale=scale 
)

viewer.add_labels(
    soma_out,
    scale=scale 
)

<Labels layer 'soma_out' at 0x178f85df0>

In [51]:
viewer.add_image(
    NU_object,
    scale=scale
)
viewer.add_labels(
    _soma_out,
    scale=scale 
)

<Labels layer '_soma_out' at 0x178d1f8b0>

In [52]:
viewer.add_labels(
    _labels_out,
    scale=scale 
)

<Labels layer '_labels_out' at 0x17942c5b0>

## DEFINE `_infer_soma` function

Based on the _prototyping_ above define the function to infer soma.  

> NOTE:  although it takes the parameters as input, they are all "hard coded" below, and the function returns the parameters in the same `defaultdict`

In [55]:
##########################
# 1. infer_soma
##########################


def _infer_soma(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer soma from linearly unmixed input.

    Parameters:
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of soma    
    """

    struct_img = _raw_soma_MCZ(in_img)

    ###################
    # PRE_PROCESSING
    ###################                         
    ################# part 1- soma

    struct_img = min_max_intensity_normalization(struct_img)
    
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    struct_img = median_filter_slice_by_slice(struct_img, size=med_filter_size)

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice( struct_img,
                                                                                sigma=gaussian_smoothing_sigma,
                                                                                truncate_range=gaussian_smoothing_truncate_range,
                                                                                )

    struct_img_non_lin = _non_linear_soma_transform_MCZ(struct_img)

    ################# part 2 - nuclei
    nuclei = min_max_intensity_normalization(in_img[NUC_CH].copy() )

    med_filter_size = 4   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    nuclei = median_filter_slice_by_slice( 
                                                                    nuclei,
                                                                    size=med_filter_size  )

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
                                                                                                sigma=gaussian_smoothing_sigma,
                                                                                                truncate_range = gaussian_smoothing_truncate_range
                                                                                                )


    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D capable
    struct_obj, _bw_low_level = MO(struct_img_non_lin, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    ################# part 2 : nuclei thresholding
    #struct_obj = struct_img > filters.threshold_li(struct_img)
    threshold_factor = 0.9 #from cellProfiler
    thresh_min = .1
    thresh_max = 1.
    nuclei_object = apply_log_li_threshold(nuclei, threshold_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)


    hole_width = 5  
    # # wrapper to remoce_small_objects
    #nuclei_object = remove_small_holes(nuclei_object, hole_width ** 2 )
    nuclei_object = hole_filling(nuclei_object, hole_min=0, hole_max=hole_width**2, fill_2d=True)

    small_object_max = 45
    nuclei_object = size_filter_2D(nuclei_object, 
                                                                min_size= small_object_max**2, 
                                                                connectivity=1)


    nuclei_labels = label(nuclei_object)
    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 40  
    struct_obj = hole_filling(struct_obj, hole_min =0 , hole_max=hole_max**2, fill_2d = True) 
    #struct_obj = remove_small_holes(struct_obj, hole_max ** 2 )


    small_object_max = 45
    struct_obj = size_filter_2D(struct_obj, 
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)

    labels_out = _masked_inverted_watershed(struct_img, nuclei_labels, struct_obj) # np.logical_or(struct_obj, NU_labels > 0)

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    soma_out = _choose_max_label(struct_img,labels_out)

    return soma_out




---------------------
# TEST `_infer_soma`  function defined above


##


In [56]:

SO_label =  _infer_SOMA(img_2D) 

--------------------------

# TEST `infer_soma` exported functions


##
`infer_som a` procedure

In [14]:
from infer_subc_2d.organelles.soma import infer_soma

soma_mask =  infer_SOMA(img_2D) 

In [15]:
viewer = napari.Viewer()

viewer.scale_bar.visible = True

viewer.add_labels(
    soma_mask,
    scale=scale,
    blending='additive'
)


<Labels layer 'SO_label' at 0x16f813370>